1- Initializee the dvc repository
2- commit what we did so far
3- download the data with dvc get wrapper
4- add data to dvc tracking. this line will create a file with same name as our data with .dvc format and also a .gitignore file.
5- add the tracking data and git placeholder with git command
6- create the remote locally or online. add the remote as a dvc remote.
7- push the data using dvc order and follow by a git commit to save the version control.
8- change between different version of the model and data.

In [2]:
! dvc init


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	MLOPs.ipynb



In [5]:
! git commit -m "Initialize DVC"
# download the data and save it in a new folder
! dvc get https://github.com/iterative/dataset-registry \
          get-started/data.xml -o data/data.xml


[master d9730c7] Initialize DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
                                                                            

In [6]:
! dvc add data/data.xml

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/ali/NPEMF/.dvc/cache/files/md5'| |0/? [00:00<?,   
                                                                                
!
  0%|          |Adding data/data.xml to cache         0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/ali/NPEMF/data/data0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 13.18file/s]

To track the changes with git, run:

	git add data/.gitignore data/data.xml.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [7]:
! git add data/data.xml.dvc data/.gitignore
! git commit -m "Add raw data"

[master 3560319] Add raw data
 2 files changed, 6 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/data.xml.dvc


In [8]:
# local storage
! mkdir /tmp/dvcstore
! dvc remote add -d myremote /tmp/dvcstore

# remote storage
# ! dvc remote add -d storage s3://mybucket/dvcstore

Setting 'myremote' as a default remote.


In [9]:
! dvc push
# ! dvc pull
! git commit data/data.xml.dvc -m "Dataset updates"
! git push

Pushing
!
  0% Checking cache in '/tmp/dvcstore/files/md5'|    |0/? [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '/home/ali/NPEMF/.dvc/cache/files/md5'| |0/? [00:00<?,   
                                                                                
!
  0%|          |Pushing to local                      0/1 [00:00<?,     ?file/s]
Pushing                                                                         
1 file pushed
On branch master
Your branch is ahead of 'origin/master' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvc/config

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	MLOPs.ipynb

no changes added to commit (use "git add" and/or "git comm

In [ ]:
# change to an specific version
! git checkout HEAD~1 data/data.xml.dvc
! dvc checkout